# AIAK微调llama2-7b最佳实践

本方案旨在帮助大模型开发者快速上手在百舸平台使用AIAK进行大模型训练，以llama2-7b为例演示如何使用AIAK镜像SFT大模型微调。

## 步骤一：准备模型权重

### 下载权重

安装下载工具并下载HF权重

In [ ]:
! wget https://doc.bce.baidu.com/bce-documentation/BOS/linux-bcecmd-0.4.5.zip
! apt-get install unzip
! unzip linux-bcecmd-0.4.5.zip -d ./
! ./linux-bcecmd-0.4.5/bcecmd bos sync bos:/cce-ai-datasets/huggingface.co/meta-llama/Llama-2-7b-hf /root/pfs/models/llama2-7b/hf/huggingface.co/meta-llama/Llama-2-7b-hf  --yes

### 权重格式转换
权重格式转换，将权重从HF格式转换为MegatronCore格式，提交一个权重转换任务

In [ ]:
! pip install python-dotenv
! pip install bce-python-sdk-next==100.9.19.2
! pip install future==1.0.0
! pip install pycryptodome==3.20.0

In [ ]:
# -*- coding: utf-8 -*-
import os
import time
import logging
from baidubce.services.aihc.aihc_client import AIHCClient

import baidubce.protocol
from baidubce.bce_client_configuration import BceClientConfiguration
from baidubce.auth.bce_credentials import BceCredentials

from dotenv import load_dotenv
import os

# 加载.env文件
load_dotenv()

# 获取配置信息, 从环境变量中获取,需要先在.env文件中设置环境变量
ak = os.getenv("AK")
sk = os.getenv("SK")
host=os.getenv("HOST")

client_token = 'test-aihc-' + str(int(time.time()))
logging.info('client_token: %s', client_token)

config = BceClientConfiguration(
    credentials=BceCredentials(ak, sk),
    endpoint=host,
    protocol=baidubce.protocol.HTTPS
)

aihc_client = AIHCClient(config)

# 创建任务
resourcePoolId = 'cce-e0isdmib'

command = r"""#! /bin/bash

AIAK_TRAINING_PATH=${AIAK_TRAINING_PATH:-"/workspace/AIAK-Training-LLM"}
CONVERT_CHECKPOINT_PATH="$AIAK_TRAINING_PATH/tools/convert_checkpoint"

python $CONVERT_CHECKPOINT_PATH/model.py \
    --load_platform=huggingface \
    --save_platform=mcore \
    --common_config_path=$CONVERT_CHECKPOINT_PATH/config/llama2-7b.json \
    --tensor_model_parallel_size=${TP} \
    --pipeline_model_parallel_size=${PP} \
    --load_ckpt_path=$LOAD \
    --save_ckpt_path=$SAVE \
    --no_save_optim \
    --no_load_optim"""

payload = {
            "name": "sft-llama2-7b-ck2mc-v1",
            "jobSpec": {
                "command": command,
                "image": "registry.baidubce.com/aihc-aiak/aiak-training-llm:ubuntu22.04-cu12.3-torch2.2.0-py310-bccl1.2.7.2_v2.1.1.5_release",
                "replicas": 1,
                "envs": [
                    {
                        "name": "TP",
                        "value": "1"
                    },
                    {
                        "name": "PP",
                        "value": "1"
                    },
                    {
                        "name": "LOAD",
                        "value": "/root/pfs/models/llama2-7b/hf/huggingface.co/meta-llama/Llama-2-7b-hf"
                    },
                    {
                        "name": "SAVE",
                        "value": "/root/pfs/models/llama2-7b/mcore/huggingface.co/meta-llama/Llama-2-7b-hf/tp1_pp1"
                    }
                ]
            },
            "datasources": [
                {
                    "type": "pfs",
                    "name": "pfs-oYQuh4",
                    "mountPath": "/root/pfs"
                }
            ],
            "queue": "default",
            "priority": "normal",
            "jobFramework": "PyTorchJob"
        }
res = aihc_client.create_aijob(
    client_token=client_token,
    resourcePoolId=resourcePoolId,
    payload=payload
)
print('res', res)

## 步骤二：准备预训练数据

### 下载数据集

In [ ]:
! wget -o /root/pfs/datasets/WuDaoCorpus2.0_base_sample.tgz https://cce-ai-datasets.cdn.bcebos.com/datasets/aiak/WuDaoCorpus2.0_base_sample.tgz
! tar zxvf WuDaoCorpus2.0_base_sample.tgz -C /root/pfs/datasets

### 数据格式转换

解压缩后得到10个JSON文件，文件内容为数组对象格式：

```
[
    {
        "id": 1,
        "uniqueKey": "da73e2d0bb4e39d241c3806876621da7",
        "titleUkey": "da73e2d0bb4e39d241c3806876621da7",
        "dataType": "博客",
        "title": "引网站蜘蛛的方法",
        "content": "做站长的都希望自已做的网站被搜索引擎比如百度尽早收录。对于新站来说,蜘蛛可没不是呼之即来挥之即去的。 但是也不是一筹莫展,无计可施,只有摸透了这一只只神秘莫测的蜘蛛,有的放矢,才能随心所欲,对吧。呵呵。些话不多说,言归正传吧。 第一,蜘蛛的出动其实是非常讲究效率的,他们也懒得白跑,如果你的网站十天半个月不更新,他白跑几次后,也就不会来这么勤了。 所以,为了让蜘蛛天天来,那么就务必不让他空来,每次都喂点食。所以对策说就是最好每天更新内容了。 可以说,你规律性的多久更新一次,蜘蛛很可能也多久才来一次。 第二,尽量去掉网页上可有可无的部分吧,特别是java之类的,还有过大的图片,要尽量降低网页加载负荷,,加速网页的打开速度,网页打速度快,那么用户体验才好,跳出率才低,网页评分才高。 第三,检查内部链接结构,去除死链接和重复链接；死链接让蜘蛛原地打转,重复连接降低网页的新鲜度； 第四,尽量多从正规和相关站点获得反向链接,正规的链接能确保外链的稳定,以及免收株连；相关链接能提高外链的权重； 第五,为站点制作网站地图,包括格式和xml两种格式,作为蜘蛛爬行的向导,让蜘蛛能爬满整个网站而没有遗漏； 第六,确保服务器返回正确的数据包响应,这条比较玄,还不懂,你可以跟我说下什么意思； 第七、为每个页面制作独立的标题和meta标签(关键字、描述),这个在网页模板里写好调用代码就行； 第八、查看网页日志,监测蜘蛛的的爬行记录,蜘蛛爬行后会留下足迹,查看这些足迹就知道蜘蛛什么时候曾经光顾过这里了。 第九,直接快速的方法:用繁殖池自动繁殖引蜘蛛,它可以快速收录站群链接或者外推链接,可以实现亿万级蜘蛛网互联互串引蜘蛛,可以增加网站收录,提升网站排名。使用繁殖池引蜘蛛到其他平台发布外链不会受到种种限制。 需要繁殖池的联系官方qq: 咨询,马上为您申请开通！"
    },
    {
        "id": 2,
        "uniqueKey": "0d78638721aecbdd798a072746114d20",
        "titleUkey": "0d78638721aecbdd798a072746114d20",
        "dataType": "博客",
        "title": "千站云繁殖池收录神器是什么?",
        "content": "千站云繁殖池是一款全新的升级收录算法,全面升级,6大功能,快速收录站群链接或者外推链接,已经取代于蜘蛛池,蜘蛛池的效果差已过时了； 那什么是繁殖池？适用于什么？有什么作用？ 繁殖池是大量网站将百度蜘蛛来访时集成一个池,通过程序控制自动繁殖外链地址给蜘蛛,这样可以快速大量收录站群链接或者外推链接； 适用于医疗媒体外推、站群、泛站、目录群、寄生虫、博客、微博、论坛、b2b信息,全自动繁殖不同地址引蜘蛛,实用而操作简单,效果好,可以实现亿万级蜘蛛网互联互串引蜘蛛,可以让新站、外推链接、媒体链接等等快速增加收录,被k网站也可以尽快恢复权重和搜索引擎快照,正常收录的网站可以增加网站收录,提升网站排名让你感受到不再需要为引蜘蛛到其他平台发布外链而受到种种限制等欢喜； 如果需要可以联系官方qq: 马上申请为你开通！！！"
    }
]
```

需要先将其处理为每行一个完整对象的jsonl格式：

```
{"id":1,"uniqueKey":"da73e2d0bb4e39d241c3806876621da7","titleUkey":"da73e2d0bb4e39d241c3806876621da7","dataType":"博客","title":"引网站蜘蛛的方法","content":"做站长的都希望自已做的网站被搜索引擎比如百度尽早收录。对于新站来说,蜘蛛可没不是呼之即来挥之即去的。 但是也不是一筹莫展,无计可施,只有摸透了这一只只神秘莫测的蜘蛛,有的放矢,才能随心所欲,对吧。呵呵。些话不多说,言归正传吧。 第一,蜘蛛的出动其实是非常讲究效率的,他们也懒得白跑,如果你的网站十天半个月不更新,他白跑几次后,也就不会来这么勤了。 所以,为了让蜘蛛天天来,那么就务必不让他空来,每次都喂点食。所以对策说就是最好每天更新内容了。 可以说,你规律性的多久更新一次,蜘蛛很可能也多久才来一次。 第二,尽量去掉网页上可有可无的部分吧,特别是java之类的,还有过大的图片,要尽量降低网页加载负荷,,加速网页的打开速度,网页打速度快,那么用户体验才好,跳出率才低,网页评分才高。 第三,检查内部链接结构,去除死链接和重复链接；死链接让蜘蛛原地打转,重复连接降低网页的新鲜度； 第四,尽量多从正规和相关站点获得反向链接,正规的链接能确保外链的稳定,以及免收株连；相关链接能提高外链的权重； 第五,为站点制作网站地图,包括格式和xml两种格式,作为蜘蛛爬行的向导,让蜘蛛能爬满整个网站而没有遗漏； 第六,确保服务器返回正确的数据包响应,这条比较玄,还不懂,你可以跟我说下什么意思； 第七、为每个页面制作独立的标题和meta标签(关键字、描述),这个在网页模板里写好调用代码就行； 第八、查看网页日志,监测蜘蛛的的爬行记录,蜘蛛爬行后会留下足迹,查看这些足迹就知道蜘蛛什么时候曾经光顾过这里了。 第九,直接快速的方法:用繁殖池自动繁殖引蜘蛛,它可以快速收录站群链接或者外推链接,可以实现亿万级蜘蛛网互联互串引蜘蛛,可以增加网站收录,提升网站排名。使用繁殖池引蜘蛛到其他平台发布外链不会受到种种限制。 需要繁殖池的联系官方qq: 咨询,马上为您申请开通！"}
{"id":2,"uniqueKey":"0d78638721aecbdd798a072746114d20","titleUkey":"0d78638721aecbdd798a072746114d20","dataType":"博客","title":"千站云繁殖池收录神器是什么?","content":"千站云繁殖池是一款全新的升级收录算法,全面升级,6大功能,快速收录站群链接或者外推链接,已经取代于蜘蛛池,蜘蛛池的效果差已过时了； 那什么是繁殖池？适用于什么？有什么作用？ 繁殖池是大量网站将百度蜘蛛来访时集成一个池,通过程序控制自动繁殖外链地址给蜘蛛,这样可以快速大量收录站群链接或者外推链接； 适用于医疗媒体外推、站群、泛站、目录群、寄生虫、博客、微博、论坛、b2b信息,全自动繁殖不同地址引蜘蛛,实用而操作简单,效果好,可以实现亿万级蜘蛛网互联互串引蜘蛛,可以让新站、外推链接、媒体链接等等快速增加收录,被k网站也可以尽快恢复权重和搜索引擎快照,正常收录的网站可以增加网站收录,提升网站排名让你感受到不再需要为引蜘蛛到其他平台发布外链而受到种种限制等欢喜； 如果需要可以联系官方qq: 马上申请为你开通！！！"}
```

In [ ]:
import json
import os

# 定义要保存的jsonl文件名
output_file = '/root/pfs/datasets/datasets/aiak/WuDaoCorpus2.0_base_sample.jsonl'
input_dir = '/root/pfs/datasets/WuDaoCorpus2.0_base_sample'
# 打开文件准备写入
with open(output_file, 'w') as file:
    # 遍历当前目录下所有的.json文件
    for filename in os.listdir(input_dir):
        if filename.endswith('.json'):
            print(filename)
            # 读取json文件内容
            with open(input_dir+ '/' +filename, 'r') as json_file:
                data_list = json.load(json_file)
                # 遍历数据列表
                for data in data_list:
                    # 将字典转换为JSON字符串
                    data['text'] = data['content']
                    data['content'] = ''
                    json_str = json.dumps(data)
                    print(json_str)
                    # 写入文件，并添加换行符
                    file.write(json_str + '\n')
print(output_file)


## 步骤二：使用AIAK提交训练任务

### SFT微调模型

In [ ]:
# -*- coding: utf-8 -*-
import os
import time
import logging
from baidubce.services.aihc.aihc_client import AIHCClient

import baidubce.protocol
from baidubce.bce_client_configuration import BceClientConfiguration
from baidubce.auth.bce_credentials import BceCredentials

from dotenv import load_dotenv
import os

# 加载.env文件
load_dotenv()

# 获取配置信息, 从环境变量中获取,需要先在.env文件中设置环境变量
ak = os.getenv("AK")
sk = os.getenv("SK")
host=os.getenv("HOST")

client_token = 'test-aihc-' + str(int(time.time()))
logging.info('client_token: %s', client_token)

config = BceClientConfiguration(
    credentials=BceCredentials(ak, sk),
    endpoint=host,
    protocol=baidubce.protocol.HTTPS
)

aihc_client = AIHCClient(config)

# 创建任务
resourcePoolId = 'cce-e0isdmib'

payload ={
            "queue": "default",
            "priority": "normal",
            "jobFramework": "PyTorchJob",
            "name": "sft-llama2-7b-train-v1",
            "jobSpec": {
                "command": "bash /workspace/AIAK-Training-LLM/examples/llama2/finetuning/sft_llama2_7b.sh",
                "image": "registry.baidubce.com/aihc-aiak/aiak-training-llm:ubuntu22.04-cu12.3-torch2.2.0-py310-bccl1.2.7.2_v2.1.1.5_release",
                "replicas": 1,
                "resources": [
                    {
                        "name": "baidu.com/a800_80g_cgpu",
                        "quantity": 8
                    }
                ],
                "enableRDMA": True,
                "envs": [
                    {
                        "name": "CUDA_DEVICE_MAX_CONNECTIONS",
                        "value": "1"
                    },
                    {
                        "name": "DATA_PATH",
                        "value": "/root/pfs/datasets/datasets/aiak/alpaca_zh-llama3-train.json"
                    },
                    {
                        "name": "DATA_CACHE_PATH",
                        "value": "/root/pfs/datasets/datasets/aiak/alpaca_zh-llama3-train_cache"
                    },
                    {
                        "name": "TOKENIZER_PATH",
                        "value": "/root/pfs/models/llama2-7b/hf/huggingface.co/meta-llama/Llama-2-7b-hf"
                    },
                    {
                        "name": "CHECKPOINT_PATH",
                        "value": "/root/pfs/models/llama2-7b/mcore/huggingface.co/meta-llama/Llama-2-7b-hf/tp1_pp1"
                    }
                ]
            },
            "datasources": [
                {
                    "type": "pfs",
                    "name": "pfs-oYQuh4",
                    "mountPath": "/root/pfs"
                }
            ]
        }
res = aihc_client.create_aijob(
    client_token=client_token,
    resourcePoolId=resourcePoolId,
    payload=payload
)
print('res', res)

## 步骤三：模型转换为HF格式

训练完成后需要将MegatronCore格式的模型转换为Huggingface格式，后续您可以使用转换后的Huggingface格式的模型进行服务在线部署。

In [ ]:
# -*- coding: utf-8 -*-
import os
import time
import logging
from baidubce.services.aihc.aihc_client import AIHCClient

import baidubce.protocol
from baidubce.bce_client_configuration import BceClientConfiguration
from baidubce.auth.bce_credentials import BceCredentials

from dotenv import load_dotenv
import os

# 加载.env文件
load_dotenv()

# 获取配置信息, 从环境变量中获取,需要先在.env文件中设置环境变量
ak = os.getenv("AK")
sk = os.getenv("SK")
host=os.getenv("HOST")

client_token = 'test-aihc-' + str(int(time.time()))
logging.info('client_token: %s', client_token)

config = BceClientConfiguration(
    credentials=BceCredentials(ak, sk),
    endpoint=host,
    protocol=baidubce.protocol.HTTPS
)

aihc_client = AIHCClient(config)

# 创建任务
resourcePoolId = 'cce-e0isdmib'

command = r"""#! /bin/bash

AIAK_TRAINING_PATH=${AIAK_TRAINING_PATH:-"/workspace/AIAK-Training-LLM"}
CONVERT_CHECKPOINT_PATH="$AIAK_TRAINING_PATH/tools/convert_checkpoint"
MEGATRON_PATH=${MEGATRON_PATH:-"/workspace/AIAK-Megatron"}

python $CONVERT_CHECKPOINT_PATH/model.py \
    --load_platform=mcore \
    --save_platform=huggingface \
    --common_config_path=$CONVERT_CHECKPOINT_PATH/config/llama2-7b.json \
    --tensor_model_parallel_size=${TP} \
    --pipeline_model_parallel_size=${PP} \
    --megatron_path=$MEGATRON_PATH \
    --load_ckpt_path=$LOAD \
    --save_ckpt_path=$SAVE \
    --no_save_optim \
    --no_load_optim"""

payload = {
            "name": "sft-llama2-7b-ck2hf-v1",
            "jobSpec": {
                "command": command,
                "image": "registry.baidubce.com/aihc-aiak/aiak-training-llm:ubuntu22.04-cu12.3-torch2.2.0-py310-bccl1.2.7.2_v2.1.1.5_release",
                "replicas": 1,
                "envs": [
                    {
                        "name": "TP",
                        "value": "1"
                    },
                    {
                        "name": "PP",
                        "value": "1"
                    },
                    {
                        "name": "LOAD",
                        "value": "/root/pfs/models/llama2-7b/mcore/huggingface.co/meta-llama/Llama-2-7b-hf/tp1_pp1/iter_0005000"
                    },
                    {
                        "name": "SAVE",
                        "value": "/root/pfs/models/llama2-7b/mcore/huggingface.co/meta-llama/Llama-2-7b-hf/tp1_pp1/iter_0005000_hf"
                    }
                ]
            },
            "datasources": [
                {
                    "type": "pfs",
                    "name": "pfs-oYQuh4",
                    "mountPath": "/root/pfs"
                }
            ],
            "queue": "default",
            "priority": "normal",
            "jobFramework": "PyTorchJob"
        }
res = aihc_client.create_aijob(
    client_token=client_token,
    resourcePoolId=resourcePoolId,
    payload=payload
)
print('res', res)

## 步骤四：部署及调用模型服务

### 在控制台使用以下权重部署推理服务

checkpoint在PFS中的存储位置

```
/models/llama2-7b/mcore/huggingface.co/meta-llama/Llama-2-7b-hf/tp1_pp1/iter_0005000_hf
```

### 调用推理服务

In [ ]:
! pip install openai

In [ ]:
from openai import OpenAI


if __name__ == '__main__':
    model_name = "sft-llama2-7b"  # 创建的服务名称，请求地址中的$MODEL_NAME
    client = OpenAI(
        # This is the default and can be omitted
        api_key='aihc',
        base_url="http://192.168.0.4/v1/model/sft-llama2-7b",
    )
    completion = client.chat.completions.create(
        model=model_name,
#         temperature=0.8,
#         top_p=0.8,
#         n=1,
        max_tokens=1024,
#         frequency_penalty=1.5,
#         stop=["."],
        messages=[
            {'role': 'system', 'content': "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."},
                  {'role': 'user', 'content': '你有什么能力？'}]

    )
    print(completion)
    print("\r\n")
    for choice in completion.choices:
        # 打印生成的文本,解码为中文
        content = choice.message.content
        print(content)
        # print("[" + choice.message.content + "]")
        print("\r\n")

返回消息示例：

```
ChatCompletion(id='chatcmpl-5031ab8a-d323-4ed6-a323-2219b9a877c4', choices=[Choice(finish_reason='stop', index=None, logprobs=None, message=ChatCompletionMessage(content='\n\n答案：我是一个在社交媒体上友善并且积极的人。我会尽力帮助那些需要帮助的人，并且致力于在世界上创造积极的变化。<|im_end|></s>', refusal=None, role='assistant', function_call=None, tool_calls=None))], created='2024-09-09T22:06:11.399930621+08:00', model='sft-llama2-7b', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=53, prompt_tokens=60, total_tokens=113))
```